In [1]:
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType,DoubleType,IntegerType

In [47]:
BOM=pd.DataFrame()
PO=pd.DataFrame()
Location_master=pd.DataFrame()
sale_data=pd.DataFrame()
port_delay_df=pd.DataFrame()
financial=pd.DataFrame()

In [ ]:
BOM['product_id'] = ''
BOM['component_id'] = ''
PO['component_id'] = ''
PO['product_id'] = ''
PO['supplier_name'] = ''
PO['needed_date/lead_date'] = ''
Location_master['supplier_name'] = ''
Location_master['address'] = ''
port_delay_df['location']=''
port_delay_df['delay']=''
port_delay_df['threshold/actual']=''
financial['supplier_name']=''
financial['financial_indicator']=''

In [ ]:
supplier_location=pd.merge(PO,Location_master,on=['supplier_name'],how='left')
supplier_product_mapping=pd.merge(supplier_location,BOM,on=['component_id'],how='left')
supplier_product_portrisk=pd.merge(supplier_product_mapping,port_delay_df,on=['supplier_name','address'],how='left')

In [ ]:
supplier_product_portrisk['lead_days'] = supplier_product_portrisk['creation_date'] - supplier_product_portrisk['needed_date']

supplier_product_portrisk['days_difference'] = supplier_product_portrisk['promise_day_port_delay'] - supplier_product_portrisk['predicted/actual_days(back-up)']




In [2]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("OTR").config("spark.sql.caseSensitive", "True").getOrCreate()

In [5]:
spark

In [ ]:
# df = spark.read.json('csvjson (1).json')
# Validating Type of Output
# type(df)
# spark = SparkSession.builder().getOrCreate()
# df = spark.read.json(spark.sparkContext.wholeTextFiles('csvjson (1).json').values)

In [19]:
# note : json format from database doesnt work
multiline_df = spark.read.option("multiline","true") \
      .json("port_congestion.json")
multiline_df.show() 

+--------------------+
| tbl_port_congestion|
+--------------------+
|[{0.89, 0.54, 2.3...|
+--------------------+



In [ ]:
df2 = spark.read.option("header",True) \
     .csv("port_congestion.csv")

pandas_df = df2.toPandas()
pandas_df=pandas_df[['port_id','elastic_loc_id','port_type','congestion_data_weekly_median_delay']]
pandas_df.head(2)
# elastic_loc_id,elastic_loc_id,port_type,congestion_data_weekly_median_delay,congestion_data_weekly_max_delay,congestion_data_fortnightly_median_delay
# congestion_data_fortnightly_max_delay,congestion_data_monthly_median_delay,congestion_data_monthly_median_delay
# df2 = spark.createDataFrame(pandas_df)
# df2.show(1)
schema = StructType([StructField("port_id", StringType(), True), StructField("elastic_loc_id", IntegerType(), True)
, StructField("port_type", StringType(), True),StructField("congestion_data_weekly_median_delay", StringType(), True)])
df = spark.createDataFrame(pandas_df, schema=schema)
# df.show()
# sparkDF=spark.createDataFrame(pandas_df) 
# sparkDF.printSchema()
# sparkDF.show()

In [46]:
display(spark.read.options(header='True', inferSchema='True', delimiter=',') \
 .csv("port_congestion.csv"))

DataFrame[id: int, port_id: int, port_display_name: string, elastic_loc_id: string, port_type: string, congestion_data_weekly_median_delay: double, congestion_data_weekly_max_delay: double, congestion_data_fortnightly_median_delay: double, congestion_data_fortnightly_max_delay: double, congestion_data_monthly_median_delay: double, congestion_data_monthly_max_delay: double, congestion_date: timestamp, created_at: timestamp, last_modified: timestamp]

05-01-2023

In [11]:
from dateutil import parser
import datetime
import pandas as pd
cal=parser.parse("20221211")
print()

creating date and month using

In [17]:
def create_date(x):
   return parser.parse(x)
def month_from_date(x):
    return x.month
def create_quarter(x):
    return pd.to_datetime(x.month).quarter

# val1=create_date('20221211')
# val2=month_from_date(val1)
# print(val2)
# print(create_quarter(val1))

df['date'] = df['Date'].astype(str)
df["date"] = df["date"].apply(create_date)
df["month"] = df["date"].apply(month_from_date)
df["quarter"] = df["date"].apply(create_quarter)


12
1


In [14]:
pd.to_datetime(val.month).quarter

1

In [18]:
adidas=pd.read_csv(r'D:\AlixPartners\adidas demo\raw data\adidas.csv')

In [19]:
adidas.columns

Index(['Unnamed: 0', 'Consignee City', 'Metric Tons', 'Consignee County',
       'Final Destination', 'HS Description', 'Quantity', 'Quantity Unit',
       'Country by Port of Departure', 'Weight', 'Port of Arrival', 'Date',
       'Consignee State', 'Bill of Lading Nbr.', 'State of Port of Arrival',
       'Port of Departure', 'Consignee Declared', 'Shipper Declared', 'row_id',
       'Weight unit', 'Consignee Zip Code', 'Country of Origin',
       'Shipper Declared Address'],
      dtype='object')

In [21]:
adidas['Port of Arrival'].value_counts().head(10)

LOS ANGELES,CA                              29669
CHARLESTON,SC                               24956
NEW YORK/NEWARK AREA, NEWARK, NEW JERSEY    12314
LONG BEACH,CA                               11953
SAVANNAH,GA                                  5592
HOUSTON,TX                                   2030
PORT EVERGLADES,FL                            765
BALTIMORE,MD                                  244
SEATTLE,WA                                    221
PHILADELPHIA,PA                               164
Name: Port of Arrival, dtype: int64

In [34]:
def create_date(x):
   return parser.parse(x)
def month_from_date(x):
    # print(x.month)
    return x.month
def create_quarter(x):
    print(pd.Period(x,'Q'))
    return pd.to_datetime(x.month).quarter

# val1=create_date('20220525')
# print(val1)
# val2=month_from_date(val1)
# print(val2)
# # hp2['Qtr'] = hp2['Mth'].map(lambda x: pd.Period(x,'Q'))
# print(create_quarter(val1))
df=adidas.copy()
df['date'] = df['Date'].astype(str)
df["date"] = df["date"].apply(create_date)
df["month"] = df["date"].apply(month_from_date)
df['quarter'] = [pd.Period(df.month[i], freq='M').quarter for i in range(len(df))]
# df["quarter"] = df["date"].apply(create_quarter)


2022-05-25 00:00:00
5
5
2022Q2
1


In [25]:
df["Date"].value_counts()

20220404    511
20220422    443
20220525    428
20220520    384
20220508    366
           ... 
20190903      1
20190925      1
20190923      1
20190922      1
20200528      1
Name: Date, Length: 1786, dtype: int64